In [215]:
#http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/
import numpy as np
import sklearn
from sklearn import datasets, linear_model
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,scale

import pandas as pd
%matplotlib inline  

In [216]:
TRAINING_RATE = 1
TESTING_RATE = 1 - TRAINING_RATE
MISSING_RATE = 0.5
QUERY_RATE = 0.2
# np.random.seed(10)

In [217]:
# Generate a dataset and plot it
np.random.seed(0)
#X : array of shape [n_samples, 2]
# e.g. X[0,:]  is [0.74, 0.46]
# X, y = sklearn.datasets.load_files("../dat/wine_quality") # X is 2 x 1 vector for 200 n_samples , y is 0 or 1
data = pd.read_csv("../dat/wine_quality/winequality-white.csv",sep=';')
# plt.scatter(X[:,0], X[:,1], s=40, c=y, cmap=plt.cm.Spectral)

In [218]:
def preprocessing(d):
    #Add normalization code here if necessary
    min_max_scaler = MinMaxScaler()
    #d.ix[:,0:-1] = scale(d.ix[:,0:-1])
#     d.ix[:,0:-1] = min_max_scaler.fit_transform(d.ix[:,0:-1])
    #d['quality'] = d['quality'].apply(lambda x:1.0 if x==6 else 0.0)
    d['quality'] = d['quality'].apply(lambda x: int(x) -3)
    d = d.iloc[np.random.permutation(len(d))]
    t = int(len(d) * TRAINING_RATE)
    tn_data = d.iloc[0:t,:]
    tt_data = d.iloc[t:,:]
    
    tn_X = tn_data.ix[:,0:-1]
    tn_Y = tn_data.ix[:,-1]
    tt_X = tt_data.ix[:,0:-1]
    tt_Y = tt_data.ix[:,-1]
    
    
    return tn_X,tn_Y, tt_X, tt_Y

In [219]:
# tn_X,tn_Y,tt_X,tt_Y = preprocessing(data)
X, y,tt_X,tt_Y = preprocessing(data)
X = X.reset_index(drop=True)
X.head(15)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.3,0.32,0.35,1.40,0.050,8.0,163.0,0.99244,3.24,0.42,10.7
1,7.0,0.31,0.26,7.40,0.069,28.0,160.0,0.99540,3.13,0.46,9.8
2,7.6,0.14,0.74,1.60,0.040,27.0,103.0,0.99160,3.07,0.40,10.8
3,5.0,0.29,0.54,5.70,0.035,54.0,155.0,0.98976,3.27,0.34,12.9
4,6.0,0.28,0.22,12.15,0.048,42.0,163.0,0.99570,3.20,0.46,10.1
5,9.8,0.93,0.45,8.60,0.052,34.0,187.0,0.99940,3.12,0.59,10.2
6,8.3,0.20,0.74,4.45,0.044,33.0,130.0,0.99240,3.25,0.42,12.2
7,7.7,0.31,0.36,4.30,0.026,15.0,87.0,0.99152,3.11,0.48,12.0
8,7.2,0.18,0.31,1.10,0.045,20.0,73.0,0.99250,3.32,0.40,10.8
9,6.2,0.25,0.47,11.60,0.048,62.0,210.0,0.99680,3.19,0.50,9.5


In [220]:
# X.ix[0,0]
X = X.as_matrix() # change type "DataFrame" (of Pandas) to numpy array

In [221]:
X

array([[  7.3 ,   0.32,   0.35, ...,   3.24,   0.42,  10.7 ],
       [  7.  ,   0.31,   0.26, ...,   3.13,   0.46,   9.8 ],
       [  7.6 ,   0.14,   0.74, ...,   3.07,   0.4 ,  10.8 ],
       ..., 
       [  7.9 ,   0.33,   0.28, ...,   3.15,   0.38,   8.8 ],
       [  8.9 ,   0.26,   0.33, ...,   3.13,   0.46,  10.8 ],
       [  7.9 ,   0.51,   0.34, ...,   3.09,   0.51,  10.  ]])

In [222]:
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = x.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2 # 200 x 2 -> (200 x 3) * (3 x 2)
    a2 = sigmoid(z2) # 200 x 2
    return np.argmax(a2, axis=1)

In [223]:
def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = X.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2
    a2 = sigmoid(z2)
    corect_probs = np.square(a2 - X) * (1/2)
    data_loss = np.sum(corect_probs)
    return (1.0/num_examples) * data_loss

In [224]:
# Helper function to plot a decision boundary.
# If you don't fully understand this function don't worry, it just generates the contour plot below.
def plot_decision_boundary(pred_func):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

In [225]:
num_examples = len(X)
nn_input_dim = 11
nn_output_dim = 11
epsilon = 0.01 #GD update
print(X.shape)

(4898, 11)


In [226]:
def sigmoid(x):                                        
    return 1 / (1 + np.exp(-x))

In [227]:
def build_model(nn_hdim, num_passes=1000, print_loss=False):
    np.random.seed() # X is 200 x 2
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim) # 2 x 3
    b1 = np.zeros((1, nn_hdim)) # 1 x 3
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim) # 3 x 2
    b2 = np.zeros((1, nn_output_dim)) # 1 x 2
    
    model = {}
    
    # Gradient Descent in batch
    for i in range(0, num_passes):
        
        # Forward propagation
        z1 = X.dot(W1) + b1 #200 x 3 -> (200 x 2) dot (2 x 3)
        a1 = sigmoid(z1) # 200 x 3
        z2 = a1.dot(W2) + b2 # 200 x 2 -> (200 x 3) * (3 x 2)
        a2 = sigmoid(z2) # 200 x 2
        # Backpropagation
        delta2 = (a2 * (1 - a2)) * (a2 - X) # 200 x 2 -> (200 x 2) * (200 x 2) 
        dW2 = (a1.T).dot(delta2) # 3 x 2 -> (3 x 200) dot (200 x 2)
        db2 = np.sum(delta2, axis=0, keepdims=True) # ? x ?
        delta1 = (a1 * (1 - a1)) * (delta2.dot(W2.T)) # 200 x 3 -> (200 x 3) * ((200 x 2) dot (2 x 3))
        dW1 = (X.T).dot(delta1) # 2 x 3 -> (2 x 200) dot (200 x 3)
        db1 = np.sum(delta1, axis=0) # ? x ?
        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        if print_loss and i % 100 == 0:
          print( "Loss after iteration %i: %f" %(i, calculate_loss(model)) )
    return model, a2

In [228]:
# Build a model with a 3-dimensional hidden layer
model, a2 = build_model(3, print_loss=True)
 
# Plot the decision boundary -> no use for autoencoder
# plot_decision_boundary(lambda x: predict(model, x))
# plt.title("Decision Boundary for hidden layer size 3")

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


Loss after iteration 0: 11163.314993
Loss after iteration 100: 11163.419115
Loss after iteration 200: 11163.471048
Loss after iteration 300: 11163.477709
Loss after iteration 400: 11163.416398
Loss after iteration 500: 11163.446638
Loss after iteration 600: 11163.308761
Loss after iteration 700: 11163.447201
Loss after iteration 800: 11163.372495
Loss after iteration 900: 11163.457688


In [229]:
a2

array([[ 1.        ,  0.00831893,  0.00317973, ...,  1.        ,
         0.25540637,  1.        ],
       [ 1.        ,  0.00831893,  0.00317973, ...,  1.        ,
         0.25540637,  1.        ],
       [ 1.        ,  0.00831893,  0.00317973, ...,  1.        ,
         0.25540637,  1.        ],
       ..., 
       [ 1.        ,  0.00831893,  0.00317973, ...,  1.        ,
         0.25540637,  1.        ],
       [ 1.        ,  0.00831893,  0.00317973, ...,  1.        ,
         0.25540637,  1.        ],
       [ 1.        ,  0.00831893,  0.00317973, ...,  1.        ,
         0.25540637,  1.        ]])

In [230]:
X

array([[  7.3 ,   0.32,   0.35, ...,   3.24,   0.42,  10.7 ],
       [  7.  ,   0.31,   0.26, ...,   3.13,   0.46,   9.8 ],
       [  7.6 ,   0.14,   0.74, ...,   3.07,   0.4 ,  10.8 ],
       ..., 
       [  7.9 ,   0.33,   0.28, ...,   3.15,   0.38,   8.8 ],
       [  8.9 ,   0.26,   0.33, ...,   3.13,   0.46,  10.8 ],
       [  7.9 ,   0.51,   0.34, ...,   3.09,   0.51,  10.  ]])

In [201]:
# np.square(a2 - X) * (1/2)